<a href="https://colab.research.google.com/github/ykato27/NLP-Mecab/blob/main/Mecab_Neologd_livedoor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MecabとGensim による類似単語検出：Livedoor News を使用

Mecabとmecab-ipadic-neologdのインストール： 

NEologd辞書の追加　:
NEologdは、Web上から得た新語に対応しており、毎週更新されるMeCab用のシステム辞書です。

mecab-python もインストール


In [1]:
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null

fatal: destination path 'mecab-ipadic-neologd' already exists and is not an empty directory.


mecab-ipadic-neologdが保存されているフォルダーを調べる。

In [2]:
!mecab-config --dicdir

/usr/lib/x86_64-linux-gnu/mecab/dic


In [3]:
!cat /etc/mecabrc

;
; Configuration file of MeCab
;
; $Id: mecabrc.in,v 1.3 2006/05/29 15:36:08 taku-ku Exp $;
;
dicdir = /var/lib/mecab/dic/debian

; userdic = /home/foo/bar/user.dic

; output-format-type = wakati
; input-buffer-size = 8192

; node-format = %m\n
; bos-format = %S\n
; eos-format = EOS\n



In [4]:
# !ln -s /etc/mecabrc /usr/local/etc/mecabrc

In [5]:
import MeCab
sent = "スマホでインスタが流行っている" 
tagger = MeCab.Tagger ()
print(tagger.parse(sent))

スマホ	名詞,一般,*,*,*,*,*
で	助詞,格助詞,一般,*,*,*,で,デ,デ
インスタ	名詞,一般,*,*,*,*,*
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
流行っ	動詞,自立,*,*,五段・ラ行,連用タ接続,流行る,ハヤッ,ハヤッ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
いる	動詞,非自立,*,*,一段,基本形,いる,イル,イル
EOS



In [6]:
!echo "SMAPのキムタクはポケモンに会いに行き、スマホを売るバイトをしている" | mecab -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd

SMAP	名詞,固有名詞,一般,*,*,*,SMAP,スマップ,スマップ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
キムタク	名詞,固有名詞,一般,*,*,*,キムタク,キムタク,キムタク
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
ポケモン	名詞,固有名詞,一般,*,*,*,Pokemon!,ポケモン,ポケモン
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
会い	名詞,一般,*,*,*,*,会意,カイイ,カイー
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
行き	動詞,自立,*,*,五段・カ行促音便,連用形,行く,イキ,イキ
、	記号,読点,*,*,*,*,、,、,、
スマホ	名詞,固有名詞,一般,*,*,*,スマホ,スマホ,スマホ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
売る	動詞,自立,*,*,五段・ラ行,基本形,売る,ウル,ウル
バイト	名詞,一般,*,*,*,*,バイト,バイト,バイト
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
し	動詞,自立,*,*,サ変・スル,連用形,する,シ,シ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
いる	動詞,非自立,*,*,一段,基本形,いる,イル,イル
EOS


ここからLivedoor News を使用しますので、以下のようにファイルをダウンロードして配置してください。

/My Drive/NLP/livedoor-news-data/topic-news.xml 

この文書が入ったMy Drive にmountします。

In [7]:
! git clone https://github.com/mashyko/NLP_Mecab
%cd NLP_Mecab
!ls 

fatal: destination path 'NLP_Mecab' already exists and is not an empty directory.
/content/NLP_Mecab
corpora		      gensim_tutor_English.ipynb  mecab_janome_BOW.ipynb
datasets	      livedoor-news-data	  Mecab_Neologd_livedoor.ipynb
fasttext_intro.ipynb  Livedoor-text		  mecab_python.ipynb
Gensim_doc2vec.ipynb  MeCab_1.ipynb		  README.md
gensim_topic.ipynb    Mecab_Intro.ipynb


分かち書きファイルの作成と保存

In [8]:
!mecab  -Owakati ./livedoor-news-data/topic-news.txt -o ./livedoor-news-data/topic-wakati.txt

gemsim向けのトークン化されたファイルの作成

In [9]:
import MeCab
tagger = MeCab.Tagger('-F\s%f[6] -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

raw = open('./livedoor-news-data/topic-news.xml', 'r')
text = open('./livedoor-news-data/topic-news_text.xml', 'w')

line = raw.readline()
while line:
    result = tagger.parse(line)
    text.write(result[1:]) # skip first \s
    line = raw.readline()

raw.close()
text.close()

In [10]:
!ls livedoor-news-data/

topic-news.model     topic-news.txt  topic-wakati.txt
topic-news_text.xml  topic-news.xml


Gensimのインストール

In [11]:
!pip install gensim

word2vec による辞書の作成

In [12]:
from gensim.models import word2vec
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

sentences = word2vec.LineSentence('./livedoor-news-data/topic-news_text.xml')
model = word2vec.Word2Vec(sentences,
                          sg=1,
                          size=100,
                          min_count=1,
                          window=10,
                          hs=1,
                          negative=0)
model.save('./livedoor-news-data/topic-news.model')


2021-07-05 13:16:05,235 : INFO : collecting all words and their counts
2021-07-05 13:16:05,237 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-07-05 13:16:05,286 : INFO : PROGRESS: at sentence #10000, processed 10576 words, keeping 4912 word types
2021-07-05 13:16:05,318 : INFO : collected 8252 word types from a corpus of 17966 raw words and 16734 sentences
2021-07-05 13:16:05,322 : INFO : Loading a fresh vocabulary
2021-07-05 13:16:05,337 : INFO : effective_min_count=1 retains 8252 unique words (100% of original 8252, drops 0)
2021-07-05 13:16:05,340 : INFO : effective_min_count=1 leaves 17966 word corpus (100% of original 17966, drops 0)
2021-07-05 13:16:05,364 : INFO : deleting the raw counts dictionary of 8252 items
2021-07-05 13:16:05,367 : INFO : sample=0.001 downsamples 13 most-common words
2021-07-05 13:16:05,370 : INFO : downsampling leaves estimated 10314 word corpus (57.4% of prior 17966)
2021-07-05 13:16:05,375 : INFO : constructing a huffman

類似の単語を20個検出します。ここでは、「嵐」

In [21]:
model = word2vec.Word2Vec.load('./livedoor-news-data/topic-news.model')
model.wv.most_similar(positive=['嵐'], topn=20)

2021-07-05 13:21:34,712 : INFO : loading Word2Vec object from ./livedoor-news-data/topic-news.model
2021-07-05 13:21:34,873 : INFO : loading wv recursively from ./livedoor-news-data/topic-news.model.wv.* with mmap=None
2021-07-05 13:21:34,875 : INFO : setting ignored attribute vectors_norm to None
2021-07-05 13:21:34,877 : INFO : loading vocabulary recursively from ./livedoor-news-data/topic-news.model.vocabulary.* with mmap=None
2021-07-05 13:21:34,879 : INFO : loading trainables recursively from ./livedoor-news-data/topic-news.model.trainables.* with mmap=None
2021-07-05 13:21:34,881 : INFO : setting ignored attribute cum_table to None
2021-07-05 13:21:34,883 : INFO : loaded ./livedoor-news-data/topic-news.model
2021-07-05 13:21:34,884 : INFO : precomputing L2-norms of word weight vectors


KeyError: ignored

以上